<a href="https://colab.research.google.com/github/hyona-yu/HAR_python/blob/master/har_checking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import pandas as pd
import random

In [ ]:
! git clone https://github.com/hyona-yu/Dataset.git


Cloning into 'Dataset'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 182 (delta 1), reused 19 (delta 1), pack-reused 161
Receiving objects: 100% (182/182), 133.20 MiB | 24.24 MiB/s, done.
Resolving deltas: 100% (118/118), done.
Checking out files: 100% (163/163), done.


In [ ]:
url = 'Dataset/UCI HAR Dataset/'

In [ ]:
feature_name_df = pd.read_csv(url + 'features.txt', sep='\s+', header=None, 
                              names=['column_index','column_name'])

feature_name = feature_name_df.iloc[:,1].values.tolist()
train_csv = pd.read_csv(url + 'train/X_train.txt', sep ='\s+', header = None)
train_csv.columns = feature_name
test_csv = pd.read_csv(url + 'test/X_test.txt', sep = '\s+', header= None)
test_csv.columns = feature_name
train_y = pd.read_csv(url + 'train/y_train.txt', header = None, names= ['label'])
test_y =pd.read_csv(url + 'test/y_test.txt', header =None, names= ['label'])

In [ ]:
train_csv

In [ ]:
print(len(train_csv), len(train_y))


In [ ]:
test_y


In [ ]:
feature_name_df.head()

In [ ]:
train_y.describe()
test_y.describe()

#preprocessing version

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
new_train_csv = scaler.fit_transform(np.array(train_csv), np.array(train_y))
new_test_csv = scaler.transform(np.array(test_csv))

#SVM

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(random_state = 42)
params = {
    'C' : [0.5,1,10]
}
cv = GridSearchCV(clf, param_grid = params)
cv.fit(train_csv, train_y)


In [ ]:
pred = cv.predict(test_csv)

acc = accuracy_score(pred, test_y)
print("SVM :",acc)

#CNN

In [ ]:
device ='cuda' if torch.cuda.is_available() else 'cpu'
random.seed(42)
torch.manual_seed(42)
if device =='cuda':
    torch.cuda.manual_seed_all(42)


class CNN(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(CNN, self).__init__()
# filter 수 늘리는게 중요? kernel size 크게
        self.layer1 = torch.nn.Sequential(torch.nn.Conv1d(input_size ,128, kernel_size= 1, stride= 1), torch.nn.MaxPool1d(kernel_size= 1), 
                                          torch.nn.ReLU()) #
        self.layer2 = torch.nn.Sequential(torch.nn.Conv1d(128,128, kernel_size= 1, stride= 1, padding =0),
                                          torch.nn.MaxPool1d(kernel_size= 1, stride = 1, padding = 0), 
                                          torch.nn.ReLU())

        self.layer3 = torch.nn.Linear(128 , output_size, bias = True) #(input_size - kernel_size + 2*padding_size)/stride + 1

        torch.nn.init.kaiming_uniform(self.layer3.weight)

    def forward(self, x):
        out = self.layer1(x)
        #print(out.shape)
        out = self.layer2(out)
        #print(out.shape)
        out = out.view(out.size(0), -1)
        #print(out.shape)
        out = self.layer3(out)
        return out


In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
encoder.fit(train_y)
train_L = encoder.transform(train_y).toarray()

In [ ]:
model = CNN(train_csv.shape[1], train_L.shape[1]).to(device)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [ ]:
model

In [ ]:
train_csv
# train_y.shape
# train_y
train_csv.shape[1]
train_L.shape

In [ ]:
train_L

In [ ]:
np.array(train_csv).dtype#describe()

In [ ]:
#train_D = np.expand_dims(np.array(train_csv), axis = 2)

In [ ]:
train_D = torch.FloatTensor(np.array(train_csv))
train_L = torch.FloatTensor(np.array(train_L))

In [ ]:
import torch.utils
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_D, train_L), batch_size = 100, shuffle= True, drop_last = True)

In [ ]:
running_loss =0
for e in range(1001):
  for i , data in enumerate(train_loader):
    x, y = data
    #print(x.shape)
    x = x.resize(100,561,1).to(device)
    y = y.to(device)
    optimizer.zero_grad()
    h = model(x)
    loss = criterion(h, y)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  if (e+1) % 100 == 0:
    print('[%d] loss : %.3f' %(e, running_loss / 100))
    running_loss =0

print('fin')

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import torchvision
writer = SummaryWriter('runs/har_cnn')
dataiter = iter(train_loader)
img, labels = dataiter.next()
img_grid = torchvision.utils.make_grid(img)

#matplotlib_imshow(img_grid, one_channel = True)
writer.add_image('har_cnn', img_grid)

In [ ]:
# ! tensorboard --logdir=data/ --host localhost --port 8088
# ! tensorboard --logdir=runs

In [ ]:
PATH = './file.pt'
torch.save({
            'epoch': 1001,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,

            }, PATH)

In [ ]:
with torch.no_grad():
  test_D = torch.FloatTensor(np.array(test_csv))
  model.eval()
  pred=  model(test_D.view(-1,561,1).to(device))



In [ ]:
pred.shape

In [ ]:
pred[0]
np.array(test_y).reshape(2947,)[1]
#test_y = np.array(test_y).reshape(1, 2947)

In [ ]:
test_y

In [ ]:
acc = [pred[i].argmax()+1 == np.array(test_y).reshape(2947,)[i] for i in range(len(test_D))]
acc = np.array(acc, dtype=np.float32)
real_acc = acc.mean()
print('CNN : ', real_acc)

#LSTM

In [ ]:
import random
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [ ]:
class LSTM(torch.nn.Module):
  def __init__(self, n_class, bi):
    super(LSTM, self).__init__()

    self.lstm = torch.nn.LSTM(561, 128, 2,dropout = 0.1, bidirectional = bi)
    self.l1 = torch.nn.Linear(128, n_class)
    self.dropout = torch.nn.Dropout(0.1)
    self.tanh = torch.nn.Tanh()
    self.softmax = torch.nn.Softmax(dim = 1)
    self.h = torch.randn(2,1,128).to(device)
    self.c = torch.randn(2,1,128).to(device)
  def forward(self, x):
    #print(x.shape)
    x = x.permute(1,0,2)# 축 바꾸기. 전치행렬처럼
    #print(x.shape)

    out, hidden = self.lstm(x,(self.h,self.c) )
    #print(out.shape)
    out = self.tanh(out)
    out = self.dropout(out)
    out = out.contiguous().view(-1, 128)
    #print(out.shape)
    out = self.l1(out)
    #out = self.softmax(out)
    return out


In [ ]:
model = LSTM(train_L.shape[1], False).to(device)

In [ ]:
model

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [ ]:
running_loss =0
for e in range(1001):
  for i , data in enumerate(train_loader):
    x, y = data
    x = x.resize(1,100,561).to(device)
    y = y.to(device)
    optimizer.zero_grad()
    h = model(x)
    loss = criterion(h, y)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  if e % 100 == 0:
    print('[%d] loss : %.3f' %(e, running_loss / 100))
    running_loss =0

print('fin')

In [ ]:
test_D.shape

In [ ]:
with torch.no_grad():
  test_D = torch.FloatTensor(np.array(test_csv))
  model.eval()
  pred_lstm = model(test_D.view(1,-1,561).to(device))



In [ ]:
print(pred_lstm)

In [ ]:
acc = [pred_lstm[i].argmax()+1 == np.array(test_y).reshape(2947,)[i] for i in range(len(test_D))]
acc = np.array(acc, dtype=np.float32)
real_acc = acc.mean()
print('LSTM : ', real_acc)

#CNN + LSTM

In [ ]:
class CNN_LSTM(torch.nn.Module):
    def __init__(self, input_size, output_size, n_class, bi):
        super(CNN_LSTM, self).__init__()
        self.layer1 = torch.nn.Sequential(torch.nn.Conv1d(input_size ,128, kernel_size= 1, stride= 1, padding =0), torch.nn.ReLU()) #
        self.layer2 = torch.nn.Sequential(torch.nn.Conv1d(128,256, kernel_size= 1, stride= 1, padding =0), torch.nn.ReLU())
        self.layer3 = torch.nn.LSTM(256,  64, 2,dropout = 0.3, bidirectional = bi)
        if bi:
          self.layer3 = torch.nn.LSTM(64, 32, 2, dropout = 0.3, bidirectional = bi)
        self.bi = bi
        #self.h = torch.randn(2,1,64).to(device)
        #self.c = torch.randn(2,1,64).to(device)
        self.layer4 = torch.nn.Linear(64, n_class)
        if bi:
          self.layer = torch.nn.Linear(32, n_class)
        torch.nn.init.xavier_uniform(self.layer4.weight)
        self.dropout = torch.nn.Dropout(0.3)

    def forward(self, x):
      out = self.layer1(x)
      #print('first layer,', out.shape)
      out = self.layer2(out)
      #print('sec layer', out.shape)
      out = out.permute(2, 0, 1)
      #print('premute', out.shape)
      out, hidden = self.layer3(out)#,(self.h,self.c))
      #print('third layer', out.shape)
      out = self.dropout(out)
      out = out.contiguous().view(-1, 64)
      #print('contig', out.view)
      out = self.layer4(out)
      #print('last layer', out)

      return out



#       first layer, torch.Size([2947, 64, 1])
# sec layer torch.Size([2947, 64, 1])
# premute torch.Size([1, 2947, 64])
# third layer torch.Size([1, 2947, 64]
      


In [ ]:
model = CNN_LSTM(train_D.shape[1], train_L.shape[1], train_L.shape[1] , False).to(device)
model

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr = 1e-3)

In [ ]:
running_loss = 0
for e in range(1001):
  for i, data in enumerate(train_loader):
    x, y= data
    x = x.view(-1,561,1).to(device)
    y = y.to(device)
    optimizer.zero_grad()
    h = model(x)
    loss = criterion(h, y)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  if e % 100 ==0:
    print('[%d] loss : %.3f' %(e, running_loss/100))
    running_loss =0
print('fin')

In [ ]:
with torch.no_grad():
  test_D = torch.FloatTensor(np.array(test_csv))
  model.eval()
  pred_cnn_lstm = model(test_D.reshape(-1,561,1).to(device))

In [ ]:
acc = [pred_cnn_lstm[i].argmax()+1 == np.array(test_y).reshape(2947,)[i] for i in range(len(test_D))]
acc = np.array(acc, dtype=np.float32)
real_acc = acc.mean()
print('LSTM_CNN : ', real_acc)

In [ ]:
acc = [pred_cnn_lstm[i].argmax()+1 == np.array(test_y).reshape(2947,)[i] for i in range(len(test_D))]
acc = np.array(acc, dtype=np.float32)
real_acc = acc.mean()
print('LSTM_CNN : ', real_acc)

#bi-LSTM_CNN

In [ ]:
model = CNN_LSTM(train_D.shape[1], train_L.shape[1], train_L.shape[1] , True).to(device)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr = 1e-2)

In [ ]:
running_loss = 0
for e in range(1001):
  for i, data in enumerate(train_loader):
    x, y= data
    #print(x.shape)
    x = x.view(-1,561,1).to(device)
    y = y.to(device)
    optimizer.zero_grad()
    h = model(x)
    loss = criterion(h, y)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  if e % 100 ==0:
    print('[%d] loss : %.3f' %(e, running_loss/100))
    running_loss =0
print('fin')

In [ ]:
with torch.no_grad():
  test_D = torch.FloatTensor(np.array(test_csv))
  model.eval()
  pred_cnn_lstm = model(test_D.reshape(-1,561,1).to(device))

In [ ]:
acc = [pred_cnn_lstm[i].argmax()+1 == np.array(test_y).reshape(2947,)[i] for i in range(len(test_D))]
acc = np.array(acc, dtype=np.float32)
real_acc = acc.mean()
print('bi-LSTM_CNN : ', real_acc)

#LSTM + CNN
####LSTM-CNN Architecture for Human Activity Recognition [IEEE]

In [ ]:
device ='cuda' if torch.cuda.is_available() else 'cpu'
random.seed(42)
torch.manual_seed(42)
if device =='cuda':
    torch.cuda.manual_seed_all(42)

In [ ]:
url = 'Dataset/HAPT Data Set/'

In [ ]:
feature_name_df = pd.read_csv(url + 'features.txt', sep='\s+', header=None, 
                              names=['column_name'])

feature_name = feature_name_df.values
train_csv = pd.read_csv(url + 'Train/X_train.txt', sep ='\s+', header = None)
train_csv.columns = feature_name
test_csv = pd.read_csv(url + 'Test/X_test.txt', sep = '\s+', header= None)
test_csv.columns = feature_name
train_y = pd.read_csv(url + 'Train/y_train.txt', header = None, names= ['label'])
test_y =pd.read_csv(url + 'Test/y_test.txt', header =None, names= ['label'])

In [ ]:
train_csv.isnull().sum().sum()

0

In [ ]:
test_csv.isnull().sum().sum()

0

In [ ]:
for i in range(len(train_y)):
  if train_y.loc[i,'label'] >=7 :
    train_csv.drop(i, inplace =True)
    train_y.drop(i, inplace = True)

In [112]:
for i in range(len(test_y)):
  if test_y.loc[i,'label'] >=7 :
    test_csv.drop(i, inplace =True)
    test_y.drop(i, inplace = True)

In [ ]:
# from sklearn.preprocessing import OneHotEncoder
# encoder = OneHotEncoder()
# encoder.fit(train_y)
# train_L = encoder.transform(train_y).toarray()
train_L = train_y

In [ ]:
train_L.describe()

,label
count,7415.000000
mean,3.654484
std,1.742175
min,1.000000
25%,2.000000
50%,4.000000
75%,5.000000
max,6.000000


In [ ]:
train_csv

,"(tBodyAcc-Mean-1,)","(tBodyAcc-Mean-2,)","(tBodyAcc-Mean-3,)","(tBodyAcc-STD-1,)","(tBodyAcc-STD-2,)","(tBodyAcc-STD-3,)","(tBodyAcc-Mad-1,)","(tBodyAcc-Mad-2,)","(tBodyAcc-Mad-3,)","(tBodyAcc-Max-1,)","(tBodyAcc-Max-2,)","(tBodyAcc-Max-3,)","(tBodyAcc-Min-1,)","(tBodyAcc-Min-2,)","(tBodyAcc-Min-3,)","(tBodyAcc-SMA-1,)","(tBodyAcc-Energy-1,)","(tBodyAcc-Energy-2,)","(tBodyAcc-Energy-3,)","(tBodyAcc-IQR-1,)","(tBodyAcc-IQR-2,)","(tBodyAcc-IQR-3,)","(tBodyAcc-ropy-1,)","(tBodyAcc-ropy-1,)","(tBodyAcc-ropy-1,)","(tBodyAcc-ARCoeff-1,)","(tBodyAcc-ARCoeff-2,)","(tBodyAcc-ARCoeff-3,)","(tBodyAcc-ARCoeff-4,)","(tBodyAcc-ARCoeff-5,)","(tBodyAcc-ARCoeff-6,)","(tBodyAcc-ARCoeff-7,)","(tBodyAcc-ARCoeff-8,)","(tBodyAcc-ARCoeff-9,)","(tBodyAcc-ARCoeff-10,)","(tBodyAcc-ARCoeff-11,)","(tBodyAcc-ARCoeff-12,)","(tBodyAcc-Correlation-1,)","(tBodyAcc-Correlation-2,)","(tBodyAcc-Correlation-3,)",...,"(fBodyAccJerkMag-Energy-1,)","(fBodyAccJerkMag-IQR-1,)","(fBodyAccJerkMag-ropy-1,)","(fBodyAccJerkMag-MaxInds-1,)","(fBodyAccJerkMag-MeanFreq-1,)","(fBodyAccJerkMag-Skewness-1,)","(fBodyAccJerkMag-Kurtosis-1,)","(fBodyGyroMag-Mean-1,)","(fBodyGyroMag-STD-1,)","(fBodyGyroMag-Mad-1,)","(fBodyGyroMag-Max-1,)","(fBodyGyroMag-Min-1,)","(fBodyGyroMag-SMA-1,)","(fBodyGyroMag-Energy-1,)","(fBodyGyroMag-IQR-1,)","(fBodyGyroMag-ropy-1,)","(fBodyGyroMag-MaxInds-1,)","(fBodyGyroMag-MeanFreq-1,)","(fBodyGyroMag-Skewness-1,)","(fBodyGyroMag-Kurtosis-1,)","(fBodyGyroJerkMag-Mean-1,)","(fBodyGyroJerkMag-STD-1,)","(fBodyGyroJerkMag-Mad-1,)","(fBodyGyroJerkMag-Max-1,)","(fBodyGyroJerkMag-Min-1,)","(fBodyGyroJerkMag-SMA-1,)","(fBodyGyroJerkMag-Energy-1,)","(fBodyGyroJerkMag-IQR-1,)","(fBodyGyroJerkMag-ropy-1,)","(fBodyGyroJerkMag-MaxInds-1,)","(fBodyGyroJerkMag-MeanFreq-1,)","(fBodyGyroJerkMag-Skewness-1,)","(fBodyGyroJerkMag-Kurtosis-1,)","(tBodyAcc-AngleWRTGravity-1,)","(tBodyAccJerk-AngleWRTGravity-1,)","(tBodyGyro-AngleWRTGravity-1,)","(tBodyGyroJerk-AngleWRTGravity-1,)","(tXAxisAcc-AngleWRTGravity-1,)","(tYAxisAcc-AngleWRTGravity-1,)","(tZAxisAcc-AngleWRTGravity-1,)"
0,0.043580,-0.005970,-0.035054,-0.995381,-0.988366,-0.937382,-0.995007,-0.988816,-0.953325,-0.794796,-0.744893,-0.648447,0.841796,0.708440,0.651716,-0.975752,-0.999950,-0.999888,-0.998014,-0.993999,-0.991980,-0.970970,-0.547095,-0.700974,-0.622697,0.921884,-0.719483,0.342168,-0.161318,0.266049,-0.274351,0.267205,-0.020958,0.382610,-0.501748,0.512463,-0.206337,0.376778,0.435172,0.660199,...,-0.999918,-0.991736,-1.000000,-0.936508,0.349260,-0.517127,-0.801006,-0.980135,-0.961301,-0.974129,-0.956013,-0.989894,-0.980135,-0.999240,-0.992673,-0.701291,-1.000000,-0.132480,0.565697,0.363478,-0.991994,-0.990877,-0.990169,-0.992521,-0.991044,-0.991994,-0.999937,-0.990537,-0.871306,-1.000000,-0.012236,-0.314848,-0.713308,-0.112754,0.030400,-0.464761,-0.018446,-0.841559,0.179913,-0.051718
1,0.039480,-0.002131,-0.029067,-0.998348,-0.982945,-0.971273,-0.998702,-0.983315,-0.974000,-0.802537,-0.736338,-0.712415,0.838758,0.708440,0.659340,-0.987427,-0.999993,-0.999826,-0.999411,-0.998918,-0.985482,-0.973481,-0.781973,-0.534604,-0.593165,0.607435,-0.266783,0.275882,0.200417,0.131266,-0.149017,0.292436,-0.192986,0.217496,-0.089175,0.059909,-0.236609,-0.012696,-0.072711,0.578649,...,-0.999867,-0.991506,-1.000000,-0.841270,0.533688,-0.625993,-0.898311,-0.988296,-0.983313,-0.982951,-0.987406,-0.992134,-0.988296,-0.999811,-0.993996,-0.720683,-0.948718,-0.268979,-0.364219,-0.723724,-0.995857,-0.996580,-0.995671,-0.996939,-0.994436,-0.995857,-0.999981,-0.994623,-1.000000,-1.000000,0.202804,-0.603199,-0.860677,0.053477,-0.007435,-0.732626,0.703511,-0.845092,0.180261,-0.047436
2,0.039978,-0.005153,-0.022651,-0.995482,-0.977314,-0.984760,-0.996415,-0.975835,-0.985973,-0.798477,-0.736338,-0.712415,0.834002,0.705008,0.674551,-0.988528,-0.999972,-0.999719,-0.999803,-0.996898,-0.976781,-0.986754,-0.688176,-0.520514,-0.593165,0.272262,-0.056424,0.322283,-0.273292,0.037180,-0.133612,0.332487,-0.240491,0.348733,-0.195409,0.229436,-0.

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# new_train_D = scaler.fit_transform(train_csv, train_L)
# new_test_D = scaler.transform(test_csv)

In [ ]:
# 이미 돼있는 dataset
import more_itertools as mit
windowed_train_D =train_csv#mit.windowed(new_train_D, n= 128, step = 128/2)


In [ ]:
np.array(train_csv).shape

(7415, 561)

In [ ]:
train_D = torch.FloatTensor(np.array(windowed_train_D))

In [ ]:
train_L = torch.LongTensor(np.array(train_y))

In [ ]:
#186
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_D, train_L), batch_size = 192, shuffle= True, drop_last = True)

In [60]:
class LSTM_CNN(torch.nn.Module):
  def __init__(self):
    super(LSTM_CNN, self).__init__()
    self.lstm1 = torch.nn.LSTM(561, 32, 2)

    self.conv1 = torch.nn.Sequential(torch.nn.Conv1d(32, 64,kernel_size = 5,stride = 2) ,torch.nn.MaxPool1d(2, stride = 2),
                                     torch.nn.ReLU())
    self.conv2 = torch.nn.Sequential( torch.nn.Conv1d(64, 128, 3, 1))
    self.gap = torch.nn.AvgPool1d(1)
    self.bn = torch.nn.BatchNorm1d(128)
    self.l1 = torch.nn.Linear(30, 7)
    self.softmax = torch.nn.Softmax(dim = 1)

  def forward(self, x):
    out, _ = self.lstm1(x)
    #print('lstm1', out.shape)
    out = out.permute(0, 2, 1)
    #print(out.shape)
    out = out.reshape(1,32,-1)
    out = self.conv1(out)
    #print('conv1', out.shape)
    out = out.reshape(1,64,-1)
    out = self.conv2(out)
    #print('conv2', out.shape)
    out = self.gap(out)
    #print('gap', out.shape)
    out = self.bn(out)
    out = out.reshape(-1,30)
    #print('batchnormal', out.shape)
    out= self.l1(out)
    #out =softmax(out)
    #print('linear', out.shape)

    return out

In [61]:
#! CUDA_LAUNCH_BLOCKING=1

In [62]:
model = LSTM_CNN().to(device)

In [63]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr = 1e-3)

In [64]:
running_loss = 0
for e in range(200):
  for i, data in enumerate(train_loader):
    x, y= data
    #print(x.shape)
    x = x.view(-1,1,561).to(device)
    y = y.flatten().to(device)
    optimizer.zero_grad()
    h = model(x)
    loss = criterion(h, y)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  if e % 10 ==0:
    print('[%d] loss : %.3f' %(e, running_loss/100))
    running_loss =0
print('fin')

[0] loss : 0.747
[10] loss : 6.826
[20] loss : 6.788
[30] loss : 6.783
[40] loss : 6.783
[50] loss : 6.592
[60] loss : 4.502
[70] loss : 2.438
[80] loss : 1.625
[90] loss : 1.034
[100] loss : 0.648
[110] loss : 0.477
[120] loss : 0.373
[130] loss : 0.315
[140] loss : 0.274
[150] loss : 0.245
[160] loss : 0.231
[170] loss : 0.215
[180] loss : 0.193
[190] loss : 0.189
fin


In [65]:
PATH = './file.pt'
torch.save({
            'epoch': 200,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,

            }, PATH)

In [66]:
np.array(test_csv).shape

(3162, 561)

In [113]:
test_D = torch.FloatTensor(np.array(test_csv))
test_L = torch.LongTensor(np.array(test_y))

In [114]:
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_D,test_L),                                                                         batch_size=192,
                                         shuffle=False ,drop_last = True)

In [129]:
total = 0
correct = 0
with torch.no_grad():
  model.eval()
  for data in test_loader:
    x, y = data
    x = x.view(-1,1,561).to(device)
    y= y.to(device)
    
    pred = model(x)
    
    _, predicted= torch.max(pred.data, 1)
    total += y.size(0)
    
    correct += (predicted ==y.reshape(-1,)).sum(0).item()
print('acc:', 100 * correct/total) 

acc: 89.47916666666667
